In [8]:
#example 1
#https://github.com/kevinzg/facebook-scraper
import facebook_scraper as fs
import pandas

# get POST_ID from the URL of the post which can have the following structure:
# https://www.facebook.com/USER/posts/POST_ID
# https://www.facebook.com/groups/GROUP_ID/posts/POST_ID
#This is the Sac State post I am using
#https://www.facebook.com/sacstate/posts/pfbid02tD4AnaqmDrfFVgzdW6qguf9GBrDB9v9uHof6TY1bXtj974dX26zRfwQCRzhEbJVzl
POST_ID = "pfbid02tD4AnaqmDrfFVgzdW6qguf9GBrDB9v9uHof6TY1bXtj974dX26zRfwQCRzhEbJVzl"

# number of comments to download -- set this to True to download all comments
#FYI if you try to download thousands of comments, Facebook will likely block your IP address
MAX_COMMENTS = 100

# get the post (this gives a generator)
gen = fs.get_posts(
    post_urls=[POST_ID],
    options={"comments": MAX_COMMENTS, "progress": True}
)

# take 1st element of the generator which is the post we requested
post = next(gen)

# extract the comments part
comments = post['comments_full']

# process comments as you want...
output = {"CommentID":[],"CommentURL":[],"CommenterID":[], "Comment":[], "CommentReactions":[],"CommentReactionCount":[]}
output2={"Comment":[],"CommentReactions":[],"CommentReactionCount":[],"Replies":[]}
for comment in comments:

    # e.g. ...print them
    #print(comment)
    output['CommentID'].append(comment['comment_id'])
    output['CommentURL'].append(comment['comment_url'])
    output['CommenterID'].append(comment['comment_id'])
    output['Comment'].append(comment['comment_text'])
    output['CommentReactions'].append(comment['comment_reactions'])
    output['CommentReactionCount'].append(comment['comment_reaction_count'])

    # e.g. ...get the replies for them
    for reply in comment['replies']:
        reply=reply['comment_text']
        if not reply:
            reply="Empty"
        print(reply)
        print("\n")
        print("Next Comment")
        output2['Comment'].append(comment['comment_text'])
        output2['CommentReactions'].append(comment['comment_reactions'])
        output2['CommentReactionCount'].append(comment['comment_reaction_count'])
        output2['Replies'].append(reply)



results = pandas.DataFrame(output)
results.to_csv('example1results.csv', index=True, index_label="Index")
results2 = pandas.DataFrame(output2)
results2.to_csv('replies.csv', index=True, index_label="Index")
print("done")


c:\Users\Sam\AppData\Local\Programs\Python\Python311\Lib\site-packages\facebook_scraper\facebook_scraper.py:912: UserWarning: Facebook says 'Unsupported Browser'
  warnings.warn(f"Facebook says 'Unsupported Browser'")


  0%|          | 0/100 [00:00<?, ?it/s]

Peter Hoey Currently the scarf can only be purchased with the ticket package.


Next Comment
Peter Hoey Apologies for dropping this here but it feels too great to visit your timeline, i always enjoy what you share here on facebook but we are not friends yet, i have tried several times to send the friend request but its not going through. do you mind trying from your side?i will be happy to be friends with you.if you find this message embarrassing please pardon my manners.thank you and remain blessed


Next Comment
Idalia Mendoza Guerrero I know but Im going to be out of town! 😩


Next Comment
If you go, get me a scarf? 🤞🏼


Next Comment
Nicole Mendoza i clicked the link & it says no tickets. 😭


Next Comment
Nicole Mendoza but if I manage to snag one I will!


Next Comment
Alex-a Rosalia Scarves were temporarily sold out, which affected this specific ticket package. Working on getting more scarves and having the link back up. Timing tbd. Thank you.


Next Comment
Kevin Winter Currently

In [9]:
#example 2
from bs4 import BeautifulSoup
import requests
import pandas

# Get data by pretending we are a browser like this.
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
url = 'https://www.yelp.com/search?cflt=restaurants&find_loc=Midtown%2C+Sacramento%2C+CA'
response = requests.get(url, headers=headers)

# print(response)
# response 200 means that it is working

soup = BeautifulSoup(response.content, 'lxml')

#I am creating an empty dictionary here
output = {"Name":[],"Description":[],"Cuisine":[], "Features":[], "Price":[],"AverageReviewScore":[],"NumberOfReviews":[]}

for item in soup.select('[class*=container]'):
    try:
        # print(item)
        if item.find('h3'):
            name = item.find('h3').get_text()
            if name[0].isdigit():
             name=name[3:]
            print(name)
            description = item.find('p', attrs={'class':'css-16lklrv'})
            descriptioninstring=str(description)
            fixeddescription=descriptioninstring[24:]
            seperator = '”'
            strippeddescription = fixeddescription.split(seperator, 1)[0]
            print(strippeddescription)
            averagereviewscore = item.find('span', class_='css-gutk1c')
            averagereviewscoreinstring = str(averagereviewscore)
            fixedaverageratingscore = averagereviewscoreinstring[53:]
            seperator = '<'
            strippedaverageratingscore = fixedaverageratingscore.split(seperator, 1)[0]
            print(strippedaverageratingscore)

            #Confusion with the location both have the same span class name
            numberofreviews = item.find('span', class_='css-chan6m').get_text()
            print(numberofreviews)
            fixednumberofreviews = numberofreviews[1:]
            fixednumberofreviews=fixednumberofreviews.replace(')', '')
            print(fixednumberofreviews)

            #Alternative attempt does not always work good
            #numberofreviewstext = item.find('div', attrs={'class':'css-1qn0b6x'}).get_text()
            #if 'reviews' in numberofreviewstext:
             #separator = 'reviews'
             #fixingnumberofreviews = numberofreviewstext.split(separator, 1)[0]
             #fixingnumberofreviews = fixingnumberofreviews[fixingnumberofreviews.find(' ('):]
             #fixednumberofreviews = fixingnumberofreviews[2:]
             #print(fixednumberofreviews)
            #else:
                #fixednumberofreviews = ''


            cuisine = item.find('span', class_='css-11bijt4').get_text()
            print(cuisine)
            features = item.find_all('span', class_='raw__09f24__T4Ezm')
            featuresinstring = str(features)
            fixedfeatures=featuresinstring.replace('<span class="raw__09f24__T4Ezm">', '')
            fixedfeatures = fixedfeatures.replace('[', '')
            fixedfeatures = fixedfeatures.replace(']', '')
            fixedfeatures = fixedfeatures.replace('</span>', '')
            print(fixedfeatures)
            price = item.find('span', class_='priceRange__09f24__mmOuH css-blvn7s')
            priceinstring = str(price)
            fixedprice = priceinstring.replace('<span class="priceRange__09f24__mmOuH css-blvn7s">', '')
            fixedprice = fixedprice.replace('</span>', '')
            print(fixedprice)
            #Append all cleaned data to the empty dictionary
            output['Name'].append(name)
            output['Description'].append(strippeddescription)
            output['Cuisine'].append(cuisine)
            output['Features'].append(fixedfeatures)
            output['Price'].append(fixedprice)
            output['AverageReviewScore'].append(strippedaverageratingscore)
            output['NumberOfReviews'].append(fixednumberofreviews)

    except Exception as e:
        raise e
        print('')

results = pandas.DataFrame(output)
results.to_csv('example2results.csv', index=True, index_label="Index")
print("done")


Pizza Twist - Downtown
This location just opened and I couldn't be happier! I called my order in and Love told me all the specials. I ordered a small butter chicken pizza. Flavors were great and tasted…
5.0 
(2 reviews)
2 reviews
Salad
Delivery, Takeout
None
Queen-Status Urban Difference Restaurant
We are brand new and now open for business and taking orders We are a soul food restaurant who does things a little bit different. We want you to come eat, enjoy and embrace your…



Soul Food
Delivery
None
Pocha House
Our bartender, Felicia was very attentive and kind! She served us quickly and welcomed us as soon as we came in :)
4.2 
(150 reviews)
150 reviews
Korean
Outdoor seating, Takeout
$$
Hidden Dumpling House Midtown
This place is REALLY good. Well worth the wait, I don't think people should complain about having to wait a long time to get a table if it's always packed. The…
3.8 
(188 reviews)
188 reviews
Chinese
Waitlist opens at 11:00 am, Outdoor seating, Delivery, Takeout
$$
Seou

In [10]:
#example 3
from bs4 import BeautifulSoup
import requests
import pandas

# Get data by pretending we are a browser like this.
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
urllist = ['https://www.yelp.com/search?cflt=restaurants&find_loc=Woodland%2C+CA','https://www.yelp.com/search?cflt=restaurants&find_loc=Woodland%2C+CA&start=10','https://www.yelp.com/search?cflt=restaurants&find_loc=Woodland%2C+CA&start=20']

#I am creating an empty dictionary here
output = {"Name":[],"Description":[],"Cuisine":[], "Features":[], "Price":[],"AverageReviewScore":[],"NumberOfReviews":[]}

for url in urllist:
 response = requests.get(url, headers=headers)

# print(response)
# response 200 means that it is working

 soup = BeautifulSoup(response.content, 'lxml')

 for item in soup.select('[class*=container]'):
    try:
        # print(item)
        if item.find('h3'):
            name = item.find('h3').get_text()
            if name[0].isdigit():
             name=name[3:]
            print(name)
            description = item.find('p', attrs={'class':'css-16lklrv'})
            descriptioninstring=str(description)
            fixeddescription=descriptioninstring[24:]
            seperator = '”'
            strippeddescription = fixeddescription.split(seperator, 1)[0]
            print(strippeddescription)
            averagereviewscore = item.find('span', class_='css-gutk1c')
            averagereviewscoreinstring = str(averagereviewscore)
            fixedaverageratingscore = averagereviewscoreinstring[53:]
            seperator = '<'
            strippedaverageratingscore = fixedaverageratingscore.split(seperator, 1)[0]
            print(strippedaverageratingscore)
            numberofreviews = item.find('span', class_='css-chan6m').get_text()
            fixednumberofreviews = numberofreviews[1:]
            fixednumberofreviews=fixednumberofreviews.replace(')', '')
            print(fixednumberofreviews)
            cuisine = item.find('span', class_='css-11bijt4').get_text()
            print(cuisine)
            features = item.find_all('span', class_='raw__09f24__T4Ezm')
            featuresinstring = str(features)
            fixedfeatures=featuresinstring.replace('<span class="raw__09f24__T4Ezm">', '')
            fixedfeatures = fixedfeatures.replace('[', '')
            fixedfeatures = fixedfeatures.replace(']', '')
            fixedfeatures = fixedfeatures.replace('</span>', '')
            print(fixedfeatures)
            price = item.find('span', class_='priceRange__09f24__mmOuH css-blvn7s')
            priceinstring = str(price)
            fixedprice = priceinstring.replace('<span class="priceRange__09f24__mmOuH css-blvn7s">', '')
            fixedprice = fixedprice.replace('</span>', '')
            print(fixedprice)
            #Append all cleaned data to the empty dictionary
            output['Name'].append(name)
            output['Description'].append(strippeddescription)
            output['Cuisine'].append(cuisine)
            output['Features'].append(fixedfeatures)
            output['Price'].append(fixedprice)
            output['AverageReviewScore'].append(strippedaverageratingscore)
            output['NumberOfReviews'].append(fixednumberofreviews)

    except Exception as e:
        raise e
        print('')

results = pandas.DataFrame(output)
results.to_csv('example3results.csv', index=True, index_label="Index")
print("done")


The Ravine on Sixteen
We, The Ravine On Sixteen are happy to welcome all visitors. We have dedicated ourselves to making sure your experience is enjoyable and memorable. We are Yolo County's new "Place to…
4.2 
46 reviews
Burgers
Outdoor seating, Delivery, Takeout
None
Aladdin’s Grill
Grand Opening May 26th! Aladdin's Grill is a delectable destination that offers an enticing array of gyros, shawarmas, and cheesesteaks, taking your taste buds on a tantalizing…
4.5 
87 reviews
Fast Food
Delivery, Takeout
None
Mojo’s Lounge & Kitchen428 Restaurant
In town for a few days and stumbled upon this spot right before last call.  We ordered vegan coconut curry and wow!  I want another bowl to take home.  It is…
4.3 
489 reviews
American (New)
Live wait time: 0 mins, Outdoor seating, Delivery, Takeout
None
Morgan’s On Main
The mock-tails are delightful and so fun to go with my kid. The atmosphere is pretty lively and I appreciate the service.
4.0 
329 reviews
American (New)
Outdoor seating, Takeou

In [19]:
#example 4
import requests
from bs4 import BeautifulSoup
import re
import pandas

r = requests.get('https://www.yelp.com/biz/i-love-teriyaki-no-7-woodland')
#r = requests.get('https://www.yelp.com/biz/pocha-house-sacramento')
soup = BeautifulSoup(r.text, 'html.parser')
#print(soup)
regex = re.compile('.*comment__09f24__D0cxf css-qgunke.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]
#print(reviews)
regex2 = re.compile('.*user-passport-info.*')
results2 = soup.find_all('div', {'class':regex2})
reviewers = [result.text for result in results2]
reviewers.pop(0)
#print(reviewers)

#Scrape review ratings
regex3 = re.compile('.*five-stars.*')
results3= soup.find_all('div', {'class':regex3})
ratings = [result['aria-label'] for result in results3]

# Replace no ratings with 0
ratings = [sub.replace('(no rating)', '0') for sub in ratings]
cleanedratings=[]
#Just keep the number portion of 5 star rating like scraped data
for rating in ratings:
    cleanedrating = rating.replace(' star rating', '')
    cleanedratings.append(cleanedrating)
#Finally in this rating list the first 5 ratings do not come from the reviews so I delete them
cleanedratings = cleanedratings[5:]
#The last few ratings in the scraped data is also wrong so delete them
cleanedratings = cleanedratings[:10]

#Check to make sure that all the lists have equal length
#print(len(reviews))
#print(len(cleanedratings))
#print(len(reviewers))

#I am creating an empty dictionary here
output = {"Reviewer":[],"Review":[],"Rating":[]}

for rating in cleanedratings:
    output['Rating'].append(rating)

for reviewer in reviewers:
    output['Reviewer'].append(reviewer)

for review in reviews:
    output['Review'].append(review)

results = pandas.DataFrame(output)
#print(results)
results.to_csv('example4results.csv', index=True, index_label="Index")
print("done")

done


In [ ]:
#example 5
#https://github.com/frizchar/yelp-reviews-scraping/blob/main/Web_Scraping.ipynb

import requests
from bs4 import BeautifulSoup
import re
import pandas

urllist = ['https://www.yelp.com/biz/pocha-house-sacramento','https://www.yelp.com/biz/pocha-house-sacramento?start=10']

#I am creating an empty dictionary here
output = {'Reviewer':[],"Review":[], 'Rating':[]}

for url in urllist:
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    #print(soup)
    #Scrape Reviews
    regex = re.compile('.*comment.*')
    results = soup.find_all('p', {'class':regex})
    reviews = [result.text for result in results]
    for review in reviews:
        output['Review'].append(review)
    #Scrape Reviewer Names
    regex2 = re.compile('.*user-passport-info.*')
    results2 = soup.find_all('div', {'class': regex2})
    reviewers = [result.text for result in results2]
    reviewers.pop(0)
    for reviewer in reviewers:
        output['Reviewer'].append(reviewer)
    #Scrape Review Ratings
    # Scrape review ratings
    regex3 = re.compile('.*five-stars.*')
    results3 = soup.find_all('div', {'class': regex3})
    ratings = [result['aria-label'] for result in results3]

    # Replace no ratings with 0
    ratings = [sub.replace('(no rating)', '0') for sub in ratings]
    cleanedratings = []
    # Just keep the number portion of 5 star rating like scraped data
    for rating in ratings:
        cleanedrating = rating.replace(' star rating', '')
        cleanedratings.append(cleanedrating)
    # Finally in this rating list the first 5 ratings do not come from the reviews so I delete them
    cleanedratings = cleanedratings[5:]
    # The last few ratings in the scraped data is also wrong so delete them
    cleanedratings = cleanedratings[:10]
    for rating in cleanedratings:
        output['Rating'].append(rating)


results = pandas.DataFrame(output)
print(results)
results.to_csv('example5results.csv', index=True, index_label="Index")
print("done")


In [ ]:
#example 6
#https://github.com/frizchar/yelp-reviews-scraping/blob/main/Web_Scraping.ipynb

import requests
from bs4 import BeautifulSoup
import re
import pandas

#I am creating an empty dictionary here
output = {'Name':[],'CommonQuestions':[]}
commonquestionslist=[]
nameslist=[]


urllist = ['https://www.yelp.com/biz/pocha-house-sacramento','https://www.yelp.com/biz/hidden-dumpling-house-midtown-sacramento','https://www.yelp.com/biz/the-rind-sacramento']

for url in urllist:
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    # Scrape most common 2 questions
    regex = re.compile('.*ux5mu6.*')
    firstquestions = soup.find_all('p', {'class': regex})[2].text
    secondquestions = soup.find_all('p', {'class': regex})[5].text
    commonquestions=firstquestions+secondquestions
    commonquestionslist.append(commonquestions)

    #Scrape Restaurant Name
    regex2 = re.compile('.*1se8maq.*')
    result3 = soup.find_all('h1', {'class': regex2})
    names=str(result3)
    names=names.replace('[<h1 class="css-1se8maq">','')
    names = names.replace('</h1>]', '')
    nameslist.append(names)

for name in nameslist:
 output['Name'].append(name)

for question in commonquestionslist:
 output['CommonQuestions'].append(question)

results = pandas.DataFrame(output)
#print(results)
results.to_csv('example6results.csv', index=True, index_label="Index")
print("done")
